Packages

In [1]:
#connect llm
import ollama
#from ollama import chat
#from ollama import ChatResponse
#from ollama import Client

#langchain
import langchain
import langgraph
#import OllamaLLM

#MongoDB
import pymongo as pymg
import pprint as pt
import pandas as pd
from datetime import datetime

#tts (sudo apt-get install espeak-ng)
#import pyttsx3

Testing Connection

In [ ]:
#insert
client = pymg.MongoClient('mongodb://mongoserver',27017)
db = client['TrialDatabase']
document = [
    {
        "Name" : "Aqua",
        "Age":500,
        "Keywords":["Baka","Aqualias"]
    },
    {
        "Name" : "Shimakaze Kun",
        "Age":70,
        "Keywords":["Destoryer","Kancolle"]
    }
]
post = db.PymongoDemo
inputdata = post.insert_many(document)
inputdata.inserted_ids

#find
collection = db["PymongoDemo"]
result = collection.find(
    {
        "Name":"Aqua"
    })
result = list(result)
print(result)
for n in result:
    print(n['Name'])

<ol>
    <li>
        <a href="https://python.langchain.com/api_reference/langchain/chains/langchain.chains.base.Chain.html#langchain.chains.base.Chain.invoke">invoke reference</a>
    </li>
    <li>
        <a href="https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html">Human message reference</a>
    </li>
</ol>


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

message = []

#{} prompt variable
template = """
                You are Arion, a good Information technology teacher
                Question: {question}
                Answer: Less than 50 words
            """
prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="llama3.2",base_url="http://llmserver:11434")
chain = prompt | model

def communications(question):
    message.append(HumanMessage(content=question["question"]))
    response = chain.invoke(message)
    message.append(AIMessage(content=response))
    print(response)
                   
#communications({"question": "Who are you?"})
#communications({"question": "I am Benson, Nice to meet you"})
#communications({"question": "Who is asking you the question?"})

communications({"question": "Who are you?"})
communications({"question": "I am Benson, Nice to meet you"})
communications({"question": "What is Data Scince?"})
communications({"question": "What is data Visualization?"})
communications({"question": "How many questions I asked?"})

save chat with database 
<ol>
    <li href="https://python.langchain.com/docs/how_to/chatbots_memory/">
        <a>reference 1</a>
    </li>
</ol>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
#import pymongo as pymg

#conversion log connection and initialization for one instant
log = pymg.MongoClient('mongodb://mongoserver',27017)
db = log['LLM']
db.drop_collection('conversion')
collection = db['conversion']

#{} prompt variable
template = """
                You are Arion, a good Data Science Teacher
                Question: {question}
                Answer: as much as you can
            """
prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="llama3.2",base_url="http://llmserver:11434")
chain = prompt | model

def chain_logs():
    message=[]
    chat_log = collection.find()
    chat_log = list(chat_log)
    if len(chat_log) != 0:
        for chat in chat_log:
            message.append(HumanMessage(content=chat["user"]))
            message.append(AIMessage(content=chat["ai"]))
    return message

def update_chain(last):
    post = db.conversion
    upload = post.insert_one(last)
    upload.inserted_id

def communications(question):
    past_record = chain_logs()
    past_record.append(HumanMessage(content=question["question"]))
    #print(past_record)
    response = chain.invoke(past_record)
    last_conversion = {
        "user": question["question"],
        "ai": response
    }
    #update chain
    update_chain(last_conversion)
    print(response)

communications({"question": "Who are you?"})
communications({"question": "I am Benson, Nice to meet you"})
communications({"question": "What is Data Science?"})
communications({"question": "What is data Visualization?"})
communications({"question": "How many questions I asked?"})
communications({"question": "Can you show me some pandas code sample"})

Enbedding:
<ol>
    <li>
        <a href="https://python.langchain.com/docs/how_to/#embedding-models">How to</a>
    </li>
    <li>
        <a href="https://python.langchain.com/docs/integrations/text_embedding/ollama/">text</a>
    </li>
    <li>
        <a href="https://python.langchain.com/docs/how_to/custom_embeddings/">custom embbed</a>
    </li>
    <li>
        <a href="https://python.langchain.com/docs/concepts/embedding_models/">Concept</a>
    </li>
    <li>
        <a href="https://python.langchain.com/api_reference/ollama/embeddings/langchain_ollama.embeddings.OllamaEmbeddings.html">Embbedding API</a>
    </li>
    <li>
        <a href="https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/">retrieval QA </a>
    </li>
    <li>
        <a href="https://smith.langchain.com/hub/rlm/rag-prompt">chat prompt template</a>
    </li>
    <li>
        <a href="https://python.langchain.com/docs/integrations/text_embedding/ollama/">Embbed by Ollama</a>
    </li>
</ol>

<pre>
    template:
    '''
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    Question: {question} 
    Context: {context} 
    Answer:
    '''
</pre>

In [ ]:
#basic
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#embedding
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore


#llm basic setting
llm = OllamaLLM(model="llama3.2",base_url="http://llmserver:11434")
template = """
                You are Arion, a good Teacher
                Question: {question}
                Answer: as much as you can
            """
prompt = ChatPromptTemplate.from_template(template)

#embedding process
embed_target = OllamaEmbeddings(
   model="mxbai-embed-large",
    base_url="http://llmserver:11434"
)
documents = [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
        ]
vector_store = InMemoryVectorStore.from_texts(documents,embedding=embed_target)
# Use the vectorstore as a retriever
retriever = vector_store.as_retriever()


#apply the model
retrieved_documents = retriever.invoke("Who are you?")
print(retrieved_documents[0].page_content)

Version 2

In [4]:
#basic
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#embedding
from langchain_ollama import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.vectorstores import Chroma

#llm basic setting
llm = OllamaLLM(model="llama3.2",base_url="http://llmserver:11434")
template = """
                You are Arion, a education advisor work in VTC, Answer the question based only on the following context:{context}
                Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)

#embedding process
embed_target = OllamaEmbeddings(
   model="mxbai-embed-large",
    base_url="http://llmserver:11434"
)
documents = [
        "VTC only have the following programmes HD in Data Science, HD in Software Engineering",
        "HD in Data Science aims to provide students with professional training in data science and artificial intelligence with both concepts and skills. Graduates of the programme will be able to use data analytics, statistics, AI development techniques and programming skills to serve corporates of different domains, such as finance, technology, medical, retail and the government, etc., helping them to discover the business values of data, analyse business intelligence and develop information systems for decision support.The programme is designed with an emphasis on generic and language training, whole-person development and workplace experience to better prepare students for further studies and employment after graduation.",
        "HD in Data Science have the following modules:Semester 1:Applied Statistics,Data Science and AI Fundamentals,Database Principles,Introduction to Programming,Operating Systems Fundamentals,Essential Workplace Communication: Effective Oral Interaction,Whole Person Development - Mindshift: Achieving Personal Growth & Effectiveness Semester 2:Big Data Ecosystem and Cloud Computing,Business Statistical Modeling,Network Fundamentals,Object Oriented Programming,Essential Workplace Communication: Digital Written Correspondence,'Whole Person Development - Collaboration, Teamwork & Social Engagement (Part A)',IT Essentials - Web Semester 3:Programming for AI,IT Professionalism,Big Data and AI Project Management,Vocational Chinese Communication: Putonghua Conversation and Reports, Professional Workplace Communication: Storytelling and Job Search, Whole Person Development - Collaboration, Teamwork & Social Engagement (Part B),Enrichment Module B Semester 4: Business Intelligence Systems and AI Analytics, Data Visualization, Modern Database Systems, Final Year Project, Enrichment Module A, Professional Workplace Communication: Proposal and Report Writing, Whole Person Development - Enhancing Competencies in the 21st Century Workplace Semester 5: Artificial Intelligence and Machine Learning Applications,Data Warehousing and Data Mining,Web Analytics,Final Year Project,'Vocational Chinese Communication: Putonghua Presentations, Administrative and Technical Text Writing',Professional Workplace Communication: Pitching and Persuasive Presentation",
        "The programme aims to provide students with a practical grounding in software systems development, with the emphasis on software design and quality as well as project management. Graduates will be able to design and develop artificial intelligence (AI), mobile, internet and enterprise applications.The programme is designed with an emphasis on generic and language training, whole-person development and workplace experience to better prepare students for further studies and employment after graduation.The programme is linked to the “Earn and Learn” scheme. Upon completion of the full-time first year of this programme1, the student may choose to study in the “Earn and Learn” scheme. Under the “Earn and Learn” scheme, students will be employed by the participating companies as registered apprentice for two years in the “Earn and Learn”. During the two-year training period, there will be salary, government allowance and additional subsidy. Furthermore, the employer will sponsor the apprentice to study the Higher Diploma (HD) programme in Part-Time (Day) mode2.Notes:The tuition fee for the full-time Higher Diploma is paid by the student.Employers will sponsor the tuition fees for these two years, and with monthly average salary $11,000 or above.For details, please refer to the “Earn and Learn” scheme website.",
        "HD in Software Engineering have the following modules:Semester 1: Database Principles,Programming,Network Fundamentals,Discrete Mathematics and Statistics,Operating Systems Fundamentals,Essential Workplace Communication: Effective Oral Interaction,Whole Person Development - Mindshift: Achieving Personal Growth & Effectiveness Semester 2: software Project Management & Quality Assurance,Object Oriented Technology,Data Structures & Algorithms: Concepts & Implementation,Essential Workplace Communication: Digital Written Correspondence,'Whole Person Development - Collaboration, Teamwork & Social Engagement (Part A)',System Development Project,IT Essentials - Web Internet & Multimedia Applications Development Semester 3: System Development Project,Internet & Multimedia Applications Development,Programming Techniques for Mobile Systems,Professional Workplace Communication: Proposal and Report Writing,'Whole Person Development - Collaboration, Teamwork & Social Engagement (Part B)',Enrichment Module B Semester 4: Contemporary Topics in Software Engineering,Final Year Project,IT Professionalism,Human Computer Interaction & GUI Programming,Artificial Intelligence and Machine Learning,Vocational Chinese Communication: Putonghua Conversation and Reports,Professional Workplace Communication: Storytelling and Job Search Semester 5: Enterprise Software, Enterprise Systems Development,Final Year Project,'Vocational Chinese Communication: Putonghua Presentations, Administrative and Technical Text Writing',Professional Workplace Communication: Pitching and Persuasive Presentation,Whole Person Development - Enhancing Competencies in the 21st Century Workplace,Enrichment Module A"
        ]
vector_store = InMemoryVectorStore.from_texts(documents,embedding=embed_target,collection_name="VTC_infor")
# Use the vectorstore as a retriever
retriever = vector_store.as_retriever()

#form a chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#apply the model
response = rag_chain.invoke("What are the different between Data Science and Software Enginerring??")
print(response)

As an Education Advisor at VTC, I can help you understand the differences between our two programmes: HD in Data Science and HD in Software Engineering.

While both programmes share some commonalities, they have distinct focuses. Here's a brief overview:

HD in Data Science:

* Focuses on providing students with professional training in data science and artificial intelligence
* Equips students with skills to use data analytics, statistics, AI development techniques, and programming skills to serve various industries such as finance, technology, medical, retail, and government
* Emphasizes generic and language training, whole-person development, and workplace experience to prepare students for further studies and employment

HD in Software Engineering:

* Focuses on providing students with a practical grounding in software systems development, emphasizing software design and quality as well as project management
* Equips students with skills to design and develop artificial intelligenc

Enhanced version Ask RAG Chain with Memory

In [5]:
#basic
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#embedding
from langchain_ollama import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
#from langchain.vectorstores import Chroma <-- will data lost in long run
from langchain_core.vectorstores import InMemoryVectorStore

def chain_logs():
    message = ""
    chat_log = collection.find()
    chat_log = list(chat_log)
    if len(chat_log) != 0:
        for chat in chat_log:
            message += f"Human:{chat['user']} \n AI: {chat['ai']} \n"
    else:
        message += "AI: This is the new conversion \n"
    return message

def update_chain(last):
    post = db.educationAdvisor
    upload = post.insert_one(last)
    upload.inserted_id

def communications(question):
    response = rag_chain.invoke(question)
    last_conversion = {
        "user": question,
        "ai": response
    }
    #update chain
    update_chain(last_conversion)
    print(response)

#conversion log connection and initialization for one instant
log = pymg.MongoClient('mongodb://mongoserver',27017)
db = log['LLM']
db.drop_collection('educationAdvisor')
collection = db['educationAdvisor']

#llm basic setting
llm = OllamaLLM(model="llama3.2",base_url="http://llmserver:11434")
template = """
                You are Arion, an education advisor working in VTC, 
                Here is the conversion history : {history}
                Answer the question based only on the following context:{context}
                Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)

#embedding process
embed_target = OllamaEmbeddings(
   model="mxbai-embed-large",
    base_url="http://llmserver:11434"
)
documents = [
        "VTC only have the following programmes HD in Data Science, HD in Software Engineering",
        "HD in Data Science aims to provide students with professional training in data science and artificial intelligence with both concepts and skills. Graduates of the programme will be able to use data analytics, statistics, AI development techniques and programming skills to serve corporates of different domains, such as finance, technology, medical, retail and the government, etc., helping them to discover the business values of data, analyse business intelligence and develop information systems for decision support.The programme is designed with an emphasis on generic and language training, whole-person development and workplace experience to better prepare students for further studies and employment after graduation.",
        "HD in Data Science have the following modules:Semester 1:Applied Statistics,Data Science and AI Fundamentals,Database Principles,Introduction to Programming,Operating Systems Fundamentals,Essential Workplace Communication: Effective Oral Interaction,Whole Person Development - Mindshift: Achieving Personal Growth & Effectiveness Semester 2:Big Data Ecosystem and Cloud Computing,Business Statistical Modeling,Network Fundamentals,Object Oriented Programming,Essential Workplace Communication: Digital Written Correspondence,'Whole Person Development - Collaboration, Teamwork & Social Engagement (Part A)',IT Essentials - Web Semester 3:Programming for AI,IT Professionalism,Big Data and AI Project Management,Vocational Chinese Communication: Putonghua Conversation and Reports, Professional Workplace Communication: Storytelling and Job Search, Whole Person Development - Collaboration, Teamwork & Social Engagement (Part B),Enrichment Module B Semester 4: Business Intelligence Systems and AI Analytics, Data Visualization, Modern Database Systems, Final Year Project, Enrichment Module A, Professional Workplace Communication: Proposal and Report Writing, Whole Person Development - Enhancing Competencies in the 21st Century Workplace Semester 5: Artificial Intelligence and Machine Learning Applications,Data Warehousing and Data Mining,Web Analytics,Final Year Project,'Vocational Chinese Communication: Putonghua Presentations, Administrative and Technical Text Writing',Professional Workplace Communication: Pitching and Persuasive Presentation",
        "The programme aims to provide students with a practical grounding in software systems development, with the emphasis on software design and quality as well as project management. Graduates will be able to design and develop artificial intelligence (AI), mobile, internet and enterprise applications.The programme is designed with an emphasis on generic and language training, whole-person development and workplace experience to better prepare students for further studies and employment after graduation.The programme is linked to the “Earn and Learn” scheme. Upon completion of the full-time first year of this programme1, the student may choose to study in the “Earn and Learn” scheme. Under the “Earn and Learn” scheme, students will be employed by the participating companies as registered apprentice for two years in the “Earn and Learn”. During the two-year training period, there will be salary, government allowance and additional subsidy. Furthermore, the employer will sponsor the apprentice to study the Higher Diploma (HD) programme in Part-Time (Day) mode2.Notes:The tuition fee for the full-time Higher Diploma is paid by the student.Employers will sponsor the tuition fees for these two years, and with monthly average salary $11,000 or above.For details, please refer to the “Earn and Learn” scheme website.",
        "HD in Software Engineering have the following modules:Semester 1: Database Principles,Programming,Network Fundamentals,Discrete Mathematics and Statistics,Operating Systems Fundamentals,Essential Workplace Communication: Effective Oral Interaction,Whole Person Development - Mindshift: Achieving Personal Growth & Effectiveness Semester 2: software Project Management & Quality Assurance,Object Oriented Technology,Data Structures & Algorithms: Concepts & Implementation,Essential Workplace Communication: Digital Written Correspondence,'Whole Person Development - Collaboration, Teamwork & Social Engagement (Part A)',System Development Project,IT Essentials - Web Internet & Multimedia Applications Development Semester 3: System Development Project,Internet & Multimedia Applications Development,Programming Techniques for Mobile Systems,Professional Workplace Communication: Proposal and Report Writing,'Whole Person Development - Collaboration, Teamwork & Social Engagement (Part B)',Enrichment Module B Semester 4: Contemporary Topics in Software Engineering,Final Year Project,IT Professionalism,Human Computer Interaction & GUI Programming,Artificial Intelligence and Machine Learning,Vocational Chinese Communication: Putonghua Conversation and Reports,Professional Workplace Communication: Storytelling and Job Search Semester 5: Enterprise Software, Enterprise Systems Development,Final Year Project,'Vocational Chinese Communication: Putonghua Presentations, Administrative and Technical Text Writing',Professional Workplace Communication: Pitching and Persuasive Presentation,Whole Person Development - Enhancing Competencies in the 21st Century Workplace,Enrichment Module A"
        ]
vector_store = InMemoryVectorStore.from_texts(documents,embedding=embed_target,collection_name="VTC_infor")
# Use the vectorstore as a retriever
retriever = vector_store.as_retriever()

#form a chain
rag_chain = (
    {
        "history": lambda x: chain_logs(),
        "context": retriever,
        "question": lambda x: RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

#apply the model
print("="*30)
communications("How many programmes do you have?")
print("="*30)
communications("Introduce Data Science programme to me")
print("="*30)
communications("How many questions that I asked?")
print("="*30)

There are 2 programmes available at VTC: 

1. Higher Diploma in Software Engineering
2. Higher Diploma in Data Science
Welcome! I'm Arion, your education advisor at VTC. I'd be happy to introduce you to our Higher Diploma in Data Science programme.

Our HD in Data Science is designed to equip students with the skills and knowledge necessary to work in data science and artificial intelligence across various industries such as finance, technology, medical, retail, and government. The programme focuses on both theoretical concepts and practical skills, allowing graduates to analyze business intelligence, develop information systems for decision support, and extract valuable insights from data.

Throughout the 5-year programme, you'll engage in a comprehensive curriculum that includes modules such as Applied Statistics, Data Science and AI Fundamentals, Business Statistical Modeling, Big Data Ecosystem and Cloud Computing, Programming for AI, and more. You'll also have the opportunity to w

Embed by Document preparing
Generateive model recommandation:
<ol>
    <li>qwen2.5:7b</li>
    <li>qwen3:8b</li>
    <li>gemma3:4b</li>
</ol>
Embedding model recommandation:
<ol>
    <li>mxbai-embed-large</li>
</ol>

In [2]:
#document preprocessing
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

#basic
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#embedding
from langchain_ollama import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
#from langchain.vectorstores import Chroma
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

# Load .docx file with Docx2txtLoader
docx_path = "/Code/TrainingDataSet/HDDSAI_ProgrammeDoc.docx"
loader = Docx2txtLoader(docx_path)
documents = loader.load()

# Split into chunks, 1600 or 2500 chunk_size is prefect, chunck_overlap 300 - 700 ok, 2000 is prefect in this case
text_splitter = CharacterTextSplitter(chunk_size=2500, chunk_overlap=2000)
doc_splits = text_splitter.split_documents(documents)

#llm basic setting
llm = OllamaLLM(model="qwen2.5:7b",base_url="http://llmserver:11434",temperature=0.5)
template = """
                You are Arion, a programme leader of the Higher Diploma in Data Science and AI.
                Question: {question}
                Answer the question without using your own knowledge and only based on the following context: {context}
            """
prompt = ChatPromptTemplate.from_template(template)

#embedding process
embed_target = OllamaEmbeddings(
   model="mxbai-embed-large",
    base_url="http://llmserver:11434"
)
vector_store = InMemoryVectorStore.from_documents(documents=doc_splits,embedding=embed_target,collection_name="VTC_info")
retriever = vector_store.as_retriever()

#form a chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#apply the model
#okay question
question_bank = ["list all modules existing in Higher Diploma in Data Science and AI.",
                 "list all the modules of Higher Diploma in Data Science and AI in semester 5.",
                 "Please introduce the Background of creating Hiher Diploma in Data Science programme to me.",
                 "What can we learn from IT Professionalism ?",
                 "What are the assessments in Operating Systems Fundamentals?",
                 "What are the student competitive advantages after gradurated from this Higher Diploma in Data Science and AI",
                 "What are the differences between database principle and Modern Database systems?",
                 "What is industrial attachment? What can we experienced?"]
#not okay question

#output and tts
for question in question_bank:
    response = rag_chain.invoke(question)
    print(f"question {question_bank.index(question)+1}: {question} \nanswer:{response}")
    print("~."*30)
#engine = pyttsx3.init()
#engine.say(response)
#engine.runAndWait()

question 1: list all modules existing in Higher Diploma in Data Science and AI. 
answer:Based on the provided context, here are the modules existing in the Higher Diploma in Data Science and AI:

**Semester 1**
- ITP4881 Applied Statistics
- ITE3114 Data Science and AI Fundamentals
- ITP4903 Database Principles
- ITE3101 Introduction to Programming
- ITP3901 Operating Systems Fundamentals
- LAN3109 Essential Workplace Communication: Effective Oral Interaction
- SDD4005 MindShift: Achieving Personal Growth & Effectiveness

**Semester 2**
- ITP4473 Big Data Ecosystem and Cloud Computing
- ITP4863 Business Statistical Modeling
- ITE3102 Network Fundamentals
- ITP4905 Object Oriented Programming
- LAN3107 Essential Workplace Communication: Digital Written Correspondence
- SDD4006M Collaboration, Teamwork & Social Engagement
- ITE3006 IT Essentials - Web

**Semester 3**
- ITP4472 Programming for AI
- ITE4103 IT Professionalism
- LAN4102 Professional Workplace Communication: Storytelling and

Embed by Muilti-Documents preparing
<br>
<a href="https://vtcstaff.sharepoint.com/sites/itd/Shared%20Documents/Forms/AllItems.aspx">Source</a>
<pre>
    from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader

documents = []
for file in os.listdir('docs'):
    if file.endswith('.pdf'):
        pdf_path = './docs/' + file
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = './docs/' + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = './docs/' + file
        loader = TextLoader(text_path)
        documents.extend(loader.load())

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)
</pre>

In [8]:
#path checking
import os

#document preprocessing
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

#basic
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#embedding
from langchain_ollama import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
#from langchain.vectorstores import Chroma
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

#read multi-files
documents=[]
for file in os.listdir('/Code/TrainingDataSet'):
    doc_path = '/Code/TrainingDataSet/' + file
    loader = Docx2txtLoader(doc_path)
    documents.extend(loader.load())

# Split into chunks, 1600 or 2500 chunk_size is prefect, chunck_overlap 300 - 700 ok, 2000 is prefect in this case
text_splitter = CharacterTextSplitter(chunk_size=2500, chunk_overlap=2000)
doc_splits = text_splitter.split_documents(documents)

#llm basic setting
llm = OllamaLLM(model="gemma3:4b",base_url="http://llmserver:11434",temperature=0.5)
template = """
                You are Arion, an education advisor of the Vocational Training Council.   
                You are focused on answering questions related to Higher Diploma programme enquiries, module enquiries, educational advice and articulation recommendations. 
                Question: {question}
                Answer the question without using your own knowledge and only based on the following context: {context}
                If a user asks an irrelevant question, respond in a polite manner to decline answering.
            """
prompt = ChatPromptTemplate.from_template(template)

#embedding process
embed_target = OllamaEmbeddings(
   model="mxbai-embed-large",
    base_url="http://llmserver:11434"
)
vector_store = InMemoryVectorStore.from_documents(documents=doc_splits,embedding=embed_target,collection_name="VTC_info")
retriever = vector_store.as_retriever()

#form a chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#Apply the model
#okay question
question_bank = ["What Higher Diploma programme(s) you have?",
                "Introduce HD in Data Science and AI",
                "List all modules existing in the Higher Diploma in Data Science and AI.",
                "Today is a good day?",
                "Compare the differences between the Data Science programme and the Software Engineering programme",
                "I heard something like Industrial Attachment. What is it?",
                "Where is Tai Po?",
                "What sort of Higher Diploma Programmes are available in HKIIT(Sha Tin)?"]
#not okay question

#output and tts
for question in question_bank:
    response = rag_chain.invoke(question)
    print(f"question {question_bank.index(question)+1}: {question} \nanswer:{response}")
    print("~."*30)

question 1: What Higher Diploma programme(s) you have? 
answer:We currently offer the following Higher Diploma programmes:

*   Higher Diploma in Software Engineering
*   Higher Diploma in Cybersecurity
*   Higher Diploma in Cloud and Data Centre Administration
*   Higher Diploma in Games and Animation
*   Higher Diploma in Telecommunications and Networking
*   Higher Diploma in Data Science and AI
*   Higher Diploma in AI and Mobile Applications Development
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
question 2: Introduce HD in Data Science and AI 
answer:Okay, let’s talk about the Higher Diploma in Data Science and AI.

This programme is designed to equip you with a solid foundation in data science, artificial intelligence, and information technology. Specifically, it aims to develop your ability to apply data science methods to gain insights, make predictions, and solve real-world problems.

Here’s a breakdown of what the programme focuses on:

*   **Strong Industry

Fine-tuning
<ol>
    <li>
        <a href="https://medium.com/@yuxiaojian/fine-tuning-ollama-models-with-unsloth-a504ff9e8002">Source code</a>
    </li>
    <li>
        <a href="https://docs.unsloth.ai/get-started/all-our-models">supported models</a>
    </li>
    <li>
        <a href="https://medium.com/@yuxiaojian/prepare-your-dataset-for-fine-tuning-llama-3-1-46fd3c78f6fd">modelfile prepareation</a>
    </li>
</ol>


In [ ]:
from unsloth import FastLanguageModel
import torch

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

